# Sysmon driver unload
Detect possible Sysmon driver unload

## Rule Content
```
- title: Sysmon driver unload
  id: 4d7cda18-1b12-4e52-b45c-d28653210df8
  status: experimental
  author: Kirill Kiryanov, oscd.community
  description: Detect possible Sysmon driver unload
  date: 2019/10/23
  modified: 2019/11/07
  references:
  - https://www.darkoperator.com/blog/2018/10/5/operating-offensively-against-sysmon
  logsource:
    product: windows
    category: process_creation
    service: null
  detection:
    selection:
      Image|endswith: \fltmc.exe
      CommandLine|contains|all:
      - unload
      - sys
    condition: selection
  falsepositives: Unknown
  level: high
  fields:
  - CommandLine
  - Details

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_path.keyword:*\\fltmc.exe AND process_command_line.keyword:*unload* AND process_command_line.keyword:*sys*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()